In [1]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/home/fomchv/.local/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/fomchv/.local/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to mak

In [2]:
import tensorflow as tf

In [3]:
!nvidia-smi

/bin/bash: /home/fomchv/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Sat May 13 04:35:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 531.61       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060         On | 00000000:01:00.0  On |                  N/A |
| 42%   35C    P5               14W / 190W|    701MiB / 12288MiB |     14%      Default |
|                                      

In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
test_dir = "dataset/COCO2020_15.voc/test"
train_dir = "dataset/COCO2020_15.voc/train"
validation_dir = "dataset/COCO2020_15.voc/val"
classes = ["person", "dog", "horse", "bird", "cat", "bear", "sheep", "cow", "apple_green", "apple_mechanic", "apple_parsha", "apple_red", "face", "Fall-Detected", "smoke"]

In [6]:
test_data = object_detector.DataLoader.from_pascal_voc(test_dir, test_dir, classes)
train_data = object_detector.DataLoader.from_pascal_voc(train_dir, train_dir, classes)
validation_data = object_detector.DataLoader.from_pascal_voc(validation_dir, validation_dir, classes)

In [7]:
spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite0', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite0/feature-vector/1', 
  hparams={'max_instances_per_image': 8000})

In [8]:
model = object_detector.create(train_data, model_spec=spec, batch_size=16, epochs=50, train_whole_model=True, validation_data=validation_data)

Epoch 1/50


2023-05-13 05:05:08.575535: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2023-05-13 05:05:08.758313: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2023-05-13 05:05:08.758380: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2023-05-13 05:05:08.764712: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2023-05-13 05:05:10.081309: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.


6566/6566 [==============================] - 1966s 296ms/step - det_loss: 0.7881 - cls_loss: 0.4969 - box_loss: 0.0058 - reg_l2_loss: 0.0701 - loss: 0.8581 - learning_rate: 0.0140 - gradient_norm: 2.4146 - val_det_loss: 0.9585 - val_cls_loss: 0.6545 - val_box_loss: 0.0061 - val_reg_l2_loss: 0.0763 - val_loss: 1.0348
Epoch 2/50
6566/6566 [==============================] - 1940s 295ms/step - det_loss: 0.7125 - cls_loss: 0.4426 - box_loss: 0.0054 - reg_l2_loss: 0.0825 - loss: 0.7951 - learning_rate: 0.0200 - gradient_norm: 1.8835 - val_det_loss: 0.7212 - val_cls_loss: 0.4434 - val_box_loss: 0.0056 - val_reg_l2_loss: 0.0879 - val_loss: 0.8091
Epoch 3/50
6566/6566 [==============================] - 1926s 293ms/step - det_loss: 0.6525 - cls_loss: 0.3978 - box_loss: 0.0051 - reg_l2_loss: 0.0922 - loss: 0.7447 - learning_rate: 0.0199 - gradient_norm: 1.7631 - val_det_loss: 0.5345 - val_cls_loss: 0.3637 - val_box_loss: 0.0034 - val_reg_l2_loss: 0.0957 - val_loss: 0.6302
Epoch 4/50
6566/6566 [==

In [9]:
model.evaluate(test_data)

44/44 [==============================] - 59s 997ms/step



{'AP': 0.44065306,
 'AP50': 0.73047996,
 'AP75': 0.47244507,
 'APs': 0.057610672,
 'APm': 0.2813835,
 'APl': 0.50935096,
 'ARmax1': 0.33374017,
 'ARmax10': 0.5188353,
 'ARmax100': 0.57885915,
 'ARs': 0.079791665,
 'ARm': 0.45437503,
 'ARl': 0.6433888,
 'AP_/person': -1.0,
 'AP_/dog': -1.0,
 'AP_/horse': -1.0,
 'AP_/bird': -1.0,
 'AP_/cat': -1.0,
 'AP_/bear': -1.0,
 'AP_/sheep': -1.0,
 'AP_/cow': -1.0,
 'AP_/apple_green': 0.4518711,
 'AP_/apple_mechanic': 0.36539876,
 'AP_/apple_parsha': 0.29094112,
 'AP_/apple_red': 0.33440575,
 'AP_/face': 0.39335617,
 'AP_/Fall-Detected': 0.42356774,
 'AP_/smoke': 0.8250307}

In [10]:
model.export(export_dir='efficientdet_lite0/efficientdet-lite0_int8', tflite_filename='efficientdet-lite0_int8.tflite')

2023-05-14 09:01:12.358024: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2023-05-14 09:01:25.886889: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 1.800 G  ops, equivalently 0.900 G  MACs


2023-05-14 09:01:29.534219: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-05-14 09:01:29.534250: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0


Estimated count of arithmetic ops: 1.800 G  ops, equivalently 0.900 G  MACs


In [11]:
config = QuantizationConfig.for_float16()

In [12]:
model.export(export_dir='efficientdet_lite0/efficientdet-lite0_float16', tflite_filename='efficientdet-lite0_float16.tflite', quantization_config=config)

2023-05-14 09:04:23.895253: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 1.800 G  ops, equivalently 0.900 G  MACs


2023-05-14 09:04:27.179234: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-05-14 09:04:27.179277: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


In [13]:
model.evaluate_tflite('efficientdet_lite0/efficientdet-lite0_int8/efficientdet-lite0_int8.tflite', test_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


2812/2812 [==============================] - 3277s 1s/step



{'AP': 0.39613867,
 'AP50': 0.6594544,
 'AP75': 0.4324993,
 'APs': 0.03299505,
 'APm': 0.22098728,
 'APl': 0.47409192,
 'ARmax1': 0.3155736,
 'ARmax10': 0.43492427,
 'ARmax100': 0.46773785,
 'ARs': 0.035416666,
 'ARm': 0.309211,
 'ARl': 0.5518053,
 'AP_/person': -1.0,
 'AP_/dog': -1.0,
 'AP_/horse': -1.0,
 'AP_/bird': -1.0,
 'AP_/cat': -1.0,
 'AP_/bear': -1.0,
 'AP_/sheep': -1.0,
 'AP_/cow': -1.0,
 'AP_/apple_green': 0.4375057,
 'AP_/apple_mechanic': 0.30638063,
 'AP_/apple_parsha': 0.15963592,
 'AP_/apple_red': 0.30335784,
 'AP_/face': 0.3708384,
 'AP_/Fall-Detected': 0.38931078,
 'AP_/smoke': 0.8059414}